## Instructions: JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [115]:
# import libraries
import pandas as pd
from pandas.io.json import json_normalize
import json
import numpy as np  # for NaNs

### 1. Find the 10 countries with the most projects

Each entry is a project and each project has a column for the country's name. If we count the number of times each country shows up, we can get to our top 10 list easily.

In [94]:
# Import data
wb = pd.read_json('data/world_bank_projects.json')

# Cross-tabulate by country
wb_top10 = pd.crosstab(index=wb['countryshortname'], 
                       columns='project count',
                       rownames = ['Countries'],
                       colnames = [''])

# Sort Descending
wb_top10 = wb_top10.sort_values('project count', ascending=False)

# Print top 10 values
wb_top10.head(10)

,project count
Countries,
China,19
Indonesia,19
Vietnam,17
India,16
"Yemen, Republic of",13
Nepal,12
Bangladesh,12
Morocco,12
Mozambique,11


### 2. Find the top 10 major project themes (using column 'mjtheme_namecode')

We're only interested in the 'mjtheme_namecode' column, which needs to be normalized to give us two rows: 'code' and 'name'. We can see that 'name' is blank for the second entry, but not for the seventh, so our counts will not be correct. 

In [138]:
# Import as JSON string
with open('data/world_bank_projects.json', 'rb') as file:
    wb_json = json.load(file)
    # using 'with' closes the file after it's loaded

# Normalize 'mjtheme_namecode' column
wb_themes = json_normalize(wb_json, 'mjtheme_namecode')

wb_themes.head(10)

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


If we look at a crosstabulation of 'name', we can see that there are 122 missing entries.

In [137]:
# Crosstabulate by 'name'
wb_themes_name = pd.crosstab(index=wb_themes['name'],
                        rownames=['Project Theme'],
                        colnames=[''],
                        columns='Project Count')

# Sort descending
wb_themes_name = wb_themes_name.sort_values('Project Count', ascending=False)

# Print top 10
wb_themes_name.head(10)

,Project Count
Project Theme,
Environment and natural resources management,223
Rural development,202
Human development,197
Public sector governance,184
Social protection and risk management,158
Financial and private sector development,130
,122
Social dev/gender/inclusion,119
Trade and integration,72


A crosstabulation of 'code' is accurate, but not all that useful if we don't know offhand what the code stands for.

In [135]:
# Crosstabulate by 'count'
wb_themes_count = pd.crosstab(index=wb_themes['code'],
                        rownames=['Project Code'],
                        colnames=[''],
                        columns='Project Count')

# Sort descending
wb_themes_count = wb_themes_count.sort_values('Project Count', ascending=False)

# Print top 10
wb_themes_count.head(10)

,Project Count
Project Code,
11,250
10,216
8,210
2,199
6,168
4,146
7,130
5,77
9,50


### 3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

We could fix the missing entries by a lookup table, but some simple sorting will help us get the answer in fewer steps.

First, we'll sort the values by 'code', and then sort each 'code' by 'name' in descending order. This will cause all of the blanks to come at the end of their respective 'code' groupings.

In [161]:
# Sort by code, then name (descending) to allow forward-filling
wb_themes_full = wb_themes.sort_values(['code', 'name'], ascending=False)

# Show a short example of the sorting by code
wb_themes_full.loc[wb_themes_full['code'] == '3', :]

,code,name
252,3,Rule of law
359,3,Rule of law
415,3,Rule of law
508,3,Rule of law
509,3,Rule of law
789,3,Rule of law
1209,3,Rule of law
1287,3,Rule of law
1302,3,Rule of law
1316,3,Rule of law


From here, we can just replace the blanks with NaNs and use forward-fill. In the above example, this means all the blanks for '3' will be filled in with what's before them: 'Rule of law'. This is exactly what we want. We can create a crosstabulation for our results and see that the top 10 counts match those from the crosstabulation by 'code' in \#2.

In [162]:
# Replace blanks with NaNs, then forward-fill NaNs
wb_themes_full = wb_themes_full.replace('', np.nan,).ffill()

# Crosstabulate by name, as in #2
wb_themes_full = pd.crosstab(index=wb_themes_full['name'],
                            rownames=['Project Theme'],
                            colnames=[''],
                            columns='Project Count')

wb_themes_full.sort_values('Project Count', ascending=False).head(10)

,Project Count
Project Theme,
Environment and natural resources management,250
Rural development,216
Human development,210
Public sector governance,199
Social protection and risk management,168
Financial and private sector development,146
Social dev/gender/inclusion,130
Trade and integration,77
Urban development,50
